In [1]:
import torch 
from torch.autograd import Variable
import numpy as np
import pandas as pd
import os
import emoji
import torch.nn.functional as F

In [2]:
df = pd.read_csv('data/train_emoji.csv', header=None)
print(df.head())
print('--' * 50)
df.info()

                                 0  1   2     3
0           never talk to me again  3 NaN   NaN
1  I am proud of your achievements  2 NaN   NaN
2   It is the worst day in my life  3 NaN   NaN
3                 Miss you so much  0 NaN   [0]
4                     food is life  4 NaN   NaN
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 4 columns):
0    132 non-null object
1    132 non-null int64
2    0 non-null float64
3    12 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ KB


In [3]:
X = df.loc[:, 0]
Y = df.loc[:, 1]
X_list = list(X)
Y_list = list(Y)
print(X.shape)
print(Y.shape)

(132,)
(132,)


In [4]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}
for i in range(5):
    print(emoji.emojize(emoji_dictionary[str(i)], use_aliases=True))

❤️
⚾
😄
😞
🍴


In [102]:
class Net(torch.nn.Module):
    def __init__(self, Tx):
        super(Net, self).__init__()
        self.lstm = torch.nn.LSTM(Tx, 128, batch_first=True, dropout=0.5, num_layers=2)
        self.fcn = torch.nn.Linear(128, 5)
    def forward(self, x):
        r_out, (a, c) = self.lstm(x)
        ret = self.fcn(r_out[:, -1, :])
#         ret = F.softmax(ret)
        return ret

In [103]:
net = Net(50)
net

Net(
  (lstm): LSTM(50, 128, num_layers=2, batch_first=True, dropout=0.5)
  (fcn): Linear(in_features=128, out_features=5, bias=True)
)

In [7]:
with open('data/glove.6B.50d.txt', encoding='utf-8') as f:
    #用来得到3个字典
    word_vec = {}
    word_inx = {}
    inx_word = {}
    j = 0
    for i in f:
        i = i.split()
        word = i[0]
        word = word.lower()
        vec = i[1:]
        word_vec[word] = vec
        word_inx[word] = j
        inx_word[j] = word
        j += 1

In [8]:
len(word_inx.keys())

400000

In [9]:
max_len = 0
for i in X_list:
    len_ = len(i.split())
    max_len = max(max_len, len_)
print(max_len)
Tx = max_len

10


In [10]:
Y_list

[3,
 2,
 3,
 0,
 4,
 0,
 3,
 2,
 3,
 1,
 3,
 3,
 1,
 3,
 2,
 3,
 2,
 3,
 1,
 2,
 3,
 0,
 2,
 2,
 2,
 1,
 4,
 2,
 2,
 4,
 0,
 3,
 4,
 2,
 0,
 3,
 2,
 2,
 3,
 4,
 2,
 2,
 0,
 2,
 3,
 0,
 3,
 2,
 4,
 3,
 0,
 3,
 3,
 3,
 4,
 2,
 1,
 1,
 1,
 2,
 3,
 1,
 0,
 0,
 0,
 3,
 4,
 4,
 2,
 2,
 1,
 2,
 0,
 3,
 2,
 2,
 0,
 0,
 3,
 1,
 2,
 1,
 2,
 2,
 4,
 3,
 3,
 2,
 4,
 0,
 0,
 0,
 3,
 3,
 3,
 2,
 0,
 1,
 2,
 3,
 0,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 4,
 1,
 1,
 3,
 3,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 0,
 4,
 0,
 3,
 3,
 4,
 4,
 1,
 4,
 3,
 0,
 2]

In [11]:
X_train = np.zeros((len(X_list), Tx, 50))
for i, sentence in enumerate(X_list):
    for index, word in enumerate(sentence.split()):
        X_train[i, index, :] = word_vec[word.lower()]

In [106]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

In [107]:
epochs = 50
for i in range(epochs):
    x = torch.FloatTensor(X_train)
    y = torch.LongTensor(Y)
    x, y = Variable(x), Variable(y)
#     print(x.shape)
#     print(y.shape)
    y_pre = net(x)
#     print(y_pre[0, :])
#     print(r_out.shape)
#     print(type(r_out))
#     print(type(y))
#     print(y[0, :])
#     print(y_pre[0, :])
#     print(y)
#     print(y_pre[0, :].shape)
    loss = loss_func(y_pre, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    y_val = net(x)
#     print(y_pre.shape)
    label = torch.argmax(y_val, 1)
#     print(label.shape)
#     print(label)
#     print(y)
#     print(sum(label == y))
    acc = sum(label == y).data.numpy() / (label.shape[0])
    print('acc = ' + str(acc*100) + '%')

acc = 31.8181818182%
acc = 32.5757575758%
acc = 32.5757575758%
acc = 31.8181818182%
acc = 34.0909090909%
acc = 31.8181818182%
acc = 34.8484848485%
acc = 38.6363636364%
acc = 35.6060606061%
acc = 37.8787878788%
acc = 36.3636363636%
acc = 39.3939393939%
acc = 42.4242424242%
acc = 47.7272727273%
acc = 50.0%
acc = 53.7878787879%
acc = 59.0909090909%
acc = 59.8484848485%
acc = 62.1212121212%
acc = 65.1515151515%
acc = 66.6666666667%
acc = 71.2121212121%
acc = 71.9696969697%
acc = 67.4242424242%
acc = 71.2121212121%
acc = 77.2727272727%
acc = 78.7878787879%
acc = 77.2727272727%
acc = 81.0606060606%
acc = 78.0303030303%
acc = 81.0606060606%
acc = 81.0606060606%
acc = 88.6363636364%
acc = 87.8787878788%
acc = 89.3939393939%
acc = 84.8484848485%
acc = 89.3939393939%
acc = 89.3939393939%
acc = 83.3333333333%
acc = 88.6363636364%
acc = 90.9090909091%
acc = 91.6666666667%
acc = 89.3939393939%
acc = 93.1818181818%
acc = 91.6666666667%
acc = 88.6363636364%
acc = 92.4242424242%
acc = 91.6666666667%
a

In [68]:
def con_x(test_sen):
    #把str变成网络输入格式
    ret = np.zeros((1, 10, 50))
    for i, word in enumerate(test_sen.split()):
        ret[:, i, :] = word_vec[word.lower()]
    return ret

In [65]:
x.shape

torch.Size([132, 10, 50])

In [109]:
test_sen = ['I want to football', 'You are good', 
            'You are not good', 'work is hard',
            'I want to eat', 
            'Please play football', 'You are wrong', 'I hate you', 'I don not hate you', 'I do not love you', 'You are not happy']
for i in test_sen:
    test_x = con_x(i)
#     print(test_x.shape)
    test_x = torch.FloatTensor(test_x)
    test_x = Variable(test_x)
    test_y = net(test_x)
    test_label = torch.argmax(test_y, 1)
    print(i, emoji.emojize(emoji_dictionary[str(test_label.data.numpy()[0])], use_aliases=True))
#     print()

I want to football ⚾
You are good ❤️
You are not good 😞
work is hard 😄
I want to eat 🍴
Please play football ⚾
You are wrong 😞
I hate you 😞
I don not hate you 😞
I do not love you ❤️
You are not happy ❤️
